In [16]:
import requests
import shutil
import subprocess
from pathlib import Path
from hashlib import md5

In [17]:
dot_sage_dir = Path.cwd() / "src" / "sage_army_knife" / "dot_sage"
tests_dot_sage_dir = Path.cwd() / "src" / "tests" / "dot_sage"

def compile_dot_sage():
    for file in dot_sage_dir.glob("*.sage"):
        res = subprocess.run(
            f"sage --preparse {file}",
            shell=True
        )
        shutil.move(str(file) + ".py", str(file).replace(".sage", ".py"))

def compile_tests_dot_sage():
    for file in tests_dot_sage_dir.glob("*.sage"):
        res = subprocess.run(
            f"sage --preparse {file}",
            shell=True
        )
        shutil.move(str(file) + ".py", str(file).replace(".sage", ".py"))

In [18]:
script_bag_path = Path.cwd() / "src" / "sage_army_knife" / "script_bag" 

def get_original_script_path(name):
    return script_bag_path / "original" / name
def get_patch_script_path(name):
    return script_bag_path / "patches" / (name + ".patch")
def get_patched_script_path(name):
    return script_bag_path / name
def get_compiled_script_path(name):
    return script_bag_path / (name + ".py")
def get_script_path(name):
    return script_bag_path / (name.replace(".sage", ".py"))

modules = []

def register_module(name, author, url, repo, digest=None):
    modules.append({
        "name": name,
        "author": author,
        "url": url,
        "repo": repo,
        "digest": digest
    })

def download_modules():
    for module in modules:
        name = module["name"]

        script_txt = requests.get(module["url"]).text

        digest = md5(script_txt.encode()).hexdigest()
        expect_digest = module["digest"]
        if expect_digest is not None and expect_digest != digest:
            raise ValueError(f"Digest mismatch, expected {expect_digest}, found {digest}")
        print(f"{name} {digest}")

        
        script_header = "\n".join([
            f'# {name}',
            f'# Author: {module["author"]}',
            f'# Source: {module["repo"]}',
            f'# URL: {module["url"]}',
            f'# md5sum: {digest}',
            '',
        ])

        with open(get_original_script_path(name), "w") as f:
            f.write(script_header + "\n" + script_txt)

def copy_original_modules():
    for module in modules:
        name = module["name"]
        shutil.copyfile(get_original_script_path(name), get_patched_script_path(name))

def save_patches():
    for module in modules:
        name = module["name"]
        res = subprocess.run(
            f"diff {get_original_script_path(name)} {get_patched_script_path(name)} > {get_patch_script_path(name)}",
            shell=True
        )

def apply_patches():
    copy_original_modules()
    for module in modules:
        name = module["name"]
        res = subprocess.run(
            f"patch {get_patched_script_path(name)} {get_patch_script_path(name)}",
            shell=True
        )

def preparse_modules():
    for module in modules:
        name = module["name"]
        if name.endswith(".sage"):
            res = subprocess.run(
                f"sage --preparse {get_patched_script_path(name)}",
                shell=True
            )
            shutil.move(get_compiled_script_path(name), get_script_path(name))

In [19]:
register_module(
    "mimoo_boneh_durfee.sage",
    "David Wong / https://twitter.com/cryptodavidw / https://github.com/mimoo",
    "https://raw.githubusercontent.com/mimoo/RSA-and-LLL-attacks/master/boneh_durfee.sage",
    "https://github.com/mimoo/RSA-and-LLL-attacks",
    "613d5772e36e5c5f4d5aa9cb36222c88"
)

register_module(
    "mimoo_dh_backdoor.sage",
    "David Wong / https://twitter.com/cryptodavidw / https://github.com/mimoo",
    "https://raw.githubusercontent.com/mimoo/Diffie-Hellman_Backdoor/master/backdoor_generator/backdoor_generator.sage",
    "https://github.com/mimoo/Diffie-Hellman_Backdoor",
    "37bc1ad725e47a4b95c1df24877991d0"
)

register_module(
    "defund_coppersmith.sage",
    "William Wang / https://twitter.com/kleptographic / https://github.com/defund",
    "https://raw.githubusercontent.com/defund/coppersmith/master/coppersmith.sage",
    "https://github.com/defund/coppersmith",
    "46b82da12e012a098ce54083d86d8a1f"
)

register_module(
    "pqlx_ec_check_param.sage",
    "David Bouman / https://mobile.twitter.com/pqlqpql / https://github.com/pqlx",
    "https://gist.githubusercontent.com/pqlx/d0bdf2d0c4a2aa400b2b52d9bd9b7b65/raw/81c36ec909a56da758c3468132e896bbf128762c/ec-param-check.sage",
    "https://gist.github.com/pqlx/d0bdf2d0c4a2aa400b2b52d9bd9b7b65",
    "4f0feb3ecb1092c58a0c7831c4c35abb"
)

register_module(
    "rbtree_babai_cvp.sage",
    "Hyunsik Jeong / https://twitter.com/RBTree_ / https://github.com/hyunsikjeong",
    "https://raw.githubusercontent.com/hyunsikjeong/LLL/master/babai/BabaiCVP.sage",
    "https://github.com/hyunsikjeong/LLL",
    "67b28b8297343eff93360c62f4e7a8dd"
)

register_module(
    "rkm_inequality_cvp.sage",
    "Gyumin Roh / https://twitter.com/rkm0959 / https://github.com/rkm0959",
    "https://raw.githubusercontent.com/rkm0959/Inequality_Solving_with_CVP/main/solver.sage",
    "https://github.com/rkm0959/Inequality_Solving_with_CVP",
    "46e38b75e35948d599e3fac3cb26f729"
)

register_module(
    "icemonster_untwister.py",
    "https://twitter.com/icemonster666 / https://github.com/icemonster",
    "https://raw.githubusercontent.com/icemonster/symbolic_mersenne_cracker/main/main.py",
    "https://github.com/icemonster/symbolic_mersenne_cracker",
    "4940b324249fd90f6462fcf2bc70b76a"
)

In [20]:
# download_modules()
# copy_original_modules()
# save_patches()
# apply_patches()
# preparse_modules()

compile_dot_sage()
compile_tests_dot_sage()

In [22]:
import pytest

pytest.main(["--disable-warnings"])